In [3]:
import numpy as np
from scipy.optimize import root_scalar
from scipy.special import beta
from scipy.special import comb
from numba import njit
import qmcpy
import time
import pandas as pd
from scipy import stats
import math
from scipy import integrate
import multiprocessing as mp

In [4]:
class mechanism:
    ''' mechanism model as dynamics for RL
    '''

    def __init__(self, harvest_time=140):

        self.alpha_L = 0.127275
        self.c_max = 130.901733
        self.K_iN = 0.12294103
        self.K_iS = 612.178130
        self.K_iX = 59.9737695
        self.K_N = 0.02000201
        self.K_O = 0.33085322
        self.K_S = 0.0430429
        self.K_SL = 0.02165744
        self.m_s = 0.02252332
        self.r_L = 0.47917813
        self.V_evap = 2.6 * 1e-03
        self.Y_cs = 0.682572
        self.Y_ls = 0.3574429
        self.Y_xn = 10.0
        self.Y_xs = 0.2385559
        self.beta_LC_max = 0.14255192
        self.mu_max = 0.3844627
        # set up the kinetic constants
        self.dt = 1  # control in every hour
        # control variable
        # F_B
        # glucose concentration in glucose feed
        self.S_F = 917
        self.harvest_time = harvest_time
        self.delta_t = 4
        self.real_measurement = [0.0, 5.0, 10.0, 23.0, 28.0, 34.0, 47.5, 55.0, 72.0, 80.0, 95.0, 102.0, 120.0, 140.0]

    def update_coefficients(self, X_f, S, N, O, C, L, F_S, V):

        self.beta_LC = self.K_iN / (self.K_iN + N) * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * O / (
                    self.K_O + O) * self.K_iX / (self.K_iX + X_f) * (1 - C / self.c_max) * self.beta_LC_max
        self.q_C = 2 * (1 - self.r_L) * self.beta_LC
        self.beta_L = self.r_L * self.beta_LC - self.K_SL * L / (L + X_f) * O / (O + self.K_O)
        self.mu = self.mu_max * S / (S + self.K_S) * self.K_iS / (self.K_iS + S) * N / (self.K_N + N) * O / (
                    self.K_O + O) / (1 + X_f / self.K_iX)
        self.q_S = 1 / self.Y_xs * self.mu + O / (O + self.K_O) * S / (
                    self.K_S + S) * self.m_s + 1 / self.Y_cs * self.q_C + 1 / self.Y_ls * self.beta_L
        self.F_B = V / 1000 * (7.14 / self.Y_xn * self.mu * X_f + 1.59 * self.q_C * X_f)
        self.D = (self.F_B + F_S) / V

    def lipid_free_cell_growth(self, X_f, V):
        dX_f = self.dt * (self.mu * X_f - (self.D - self.V_evap / V) * X_f)
        return dX_f + X_f

    def citrate_accumulation(self, X_f, C, V):
        dC = self.dt * (self.q_C * X_f - (self.D - self.V_evap / V) * C)
        return dC + C

    def lipid_accumulation(self, X_f, L, V):
        dL = self.dt * ((self.alpha_L * self.mu + self.beta_L) * X_f - (self.D - self.V_evap / V) * L)
        return dL + L

    def glucose_consumption(self, X_f, S, F_S, V):
        dS = - self.dt * (self.q_S * X_f - F_S / V * self.S_F + (self.D - self.V_evap / V) * S)
        return dS + S

    def nitrogen_consumption(self, X_f, N, V):
        dN = - self.dt * (1 / self.Y_xn * self.mu * X_f + (self.D - self.V_evap / V) * N)
        return max(dN + N, 0)

    def volume_change(self, F_S, V):
        dV = self.dt * (self.F_B + F_S - self.V_evap)
        return dV + V

    def one_step_predict(self, X_f, C, L, S, N, V, F_S, O):
        next_X_f = X_f
        next_C = C
        next_L = L
        next_S = S
        next_N = N
        next_V = V
        for i in range(int(self.delta_t / self.dt)):
            self.update_coefficients(next_X_f, next_S, next_N, O, next_C, next_L, F_S, next_V)
            next_X_f = max(self.lipid_free_cell_growth(next_X_f, next_V), 0)
            next_C = max(self.citrate_accumulation(next_X_f, next_C, next_V), 0)
            next_L = max(self.lipid_accumulation(next_X_f, next_L, next_V), 0)
            next_S = max(self.glucose_consumption(next_X_f, next_S, F_S, next_V), 0)
            next_N = max(self.nitrogen_consumption(next_X_f, next_N, next_V), 0)
            next_V = max(self.volume_change(F_S, next_V), 0.001)
        return [next_X_f, next_C, next_L, next_S, next_N, next_V]

    def predict(self, X_f, C, L, S, N, V, F_S, t):
        O = self.dissolve_oxygen(t)
        out = self.one_step_predict(X_f, C, L, S, N, V, F_S, O)
        return out

    def dissolve_oxygen(self, t):
        oxygen_data = [98.58750, 71.34750, 43.57285, 28.51669, 24.61062, 24.58139, 28.96252, 30.63478, 24.34490,
                       29.33842, 31.89646, 34.51994, 36.03211, 36.16354]

        index = 0
        time_w = -1
        for i, time in enumerate(self.real_measurement):
            if time - t > 0:
                index = i - 1
                time_w = time
                break
        return oxygen_data[index]

class bayesian_network_posterior:
    def __init__(self, the_R, tau2_R, kap_R, lam_R, mu, num_state, num_action, n_time, beta, v2):
        self.the_R = the_R
        self.tau2_R = tau2_R
        self.kap_R = kap_R
        self.lam_R = lam_R
        self.num_nodes = (num_state + num_action) * n_time
        self.num_state = num_state
        self.num_action = num_action
        self.mu = mu
        self.beta = beta
        self.v2 = v2
        self.n_time = n_time

    def posterior_sample(self, size=1, useFixed = True):
        p_beta = np.zeros(shape=(self.num_nodes,self.num_nodes, size))
        p_v2 = np.zeros(shape=(self.num_nodes, size))
        for i in range(self.num_nodes):
            for j in range(self.num_nodes):
                if self.tau2_R[i, j] != 0:
                    p_beta[i, j, ] = np.random.normal(loc=self.the_R[i,j], scale=np.sqrt(self.tau2_R[i,j]), size=size)
            gamma_rate = self.lam_R[i] / 2
            p_v2[i,] = 1 / np.random.gamma(shape=self.kap_R[i] / 2, scale=1/gamma_rate, size=size)
        if useFixed:
            p_beta = self.beta
            p_v2 = self.v2
        return (p_beta, p_v2, self.mu)  

def rescale_action(action, t, mu, sd, scale_method = "standard"):
    mu0 = mu.to_numpy().flatten()
    sample_sd = sd.to_numpy().flatten()
    a_t = sample_sd[(6 * t):(6 * t + 1)] * action + mu0[(6 * t):(6 * t + 1)]
    return a_t

def simulate(s1, theta, mua, mus, mu, sd, mean, v2_s, process_model, in_size = 10):
    s = np.zeros((in_size, 5, 14))
    r = np.zeros(in_size)
    theta_index = [0, 1, 3, 6, 7, 9, 12, 14, 18, 20, 24, 26, 30]
    for j in range(in_size):
        [X_f, C, L, S, N, V] = s1
        [X_f, C, S, N, V] = np.maximum([X_f, C, S, N, V] + np.random.normal(loc = mean[:, 0], scale = v2_s[:, 0]/100), 0)
        s[j, :, 0] = [X_f, C, S, N, V]
        for i in range(13):
            t1 = process_model.real_measurement[i]
            t2 = theta_index[i]
            action = mua[0, t2]+np.sum(theta[:, i]*(np.array([X_f, C, S, N, V])-mus[:,t2]), axis = 0)
            F_s = rescale_action(action, t2, mu, sd).item()/20+0.008
            e_mean = mean[:, i+1]
            [X_f, C, L, S, N, V] = process_model.predict(X_f, C, L, S, N, V, F_s, t1)
            [X_f, C, S, N, V] = np.maximum([X_f, C, S, N, V] + np.random.normal(loc = e_mean, scale = v2_s[:, i+1]/100), 0)
            s[j, :, i+1] = [X_f, C, S, N, V]
            r[j] = r[j] - 534.52 * F_s
        r[j] = r[j] + 1.29 * C - 15
    return [r, s]

In [5]:
def int_sin_m(x, m):
    if m == 0:
        return x
    elif m == 1:
        return 1 - np.cos(x)
    else:
        return (m - 1) / m * int_sin_m(x, m - 2) - np.cos(x) * np.sin(x) ** (
                m - 1
        ) / m

def zero_sum_projection(d):
    basis = np.array([[1.0] * i + [-i] + [0.0] * (d - i - 1) for i in range(1, d)])
    return np.array([v / np.linalg.norm(v) for v in basis])


def bm_sphere_points(n, d):
    import qmcpy
    mod = d%2
    if mod == 1:
        dim = d+1
        pairs = dim//2
    else:
        dim = d
        pairs = dim//2
    X = qmcpy.Sobol(dim, graycode=True).gen_samples(n)
    Y = np.zeros((n, dim))
    for i in range(pairs):
        R = np.sqrt(-2 * np.log(X[:, 2*i]))
        Theta = 2 * np.pi *X[:, 2*i+1]
        Y[:, 2*i] = R * np.cos(Theta)
        Y[:, 2*i+1] = R * np.sin(Theta)
    Y = Y[:, :d]
    A = np.sum(Y**2, axis = 1)
    B = np.tile(A.reshape(n, 1), (1,d))
    Y = Y/np.sqrt(B)
    return Y

def tfww_sphere_points(n, d):
    import qmcpy
    X = qmcpy.Sobol(d-1, graycode=True).gen_samples(n)
    dim = d
    mod = dim%2
    if mod == 1:
        m = dim//2
        g = np.zeros((n, m+1))
        c = np.zeros((n, m))
        g[:, 0] = 0
        g[:, m] = 1
        for j in range(m-1):
            g[:, m-j-1] = g[:, m-j]*np.power(X[:, m-j-2],2/(2*(m-j-1)+1))
            c[:, m-j-1] = np.sqrt(g[:, m-j]-g[:, m-j-1])
        c[:, 0] = np.sqrt(g[:, 1] - g[:, 0])
        Y = np.zeros((n, dim))
        Y[:, 0] = c[:, 0]*(1-2*X[:, m-1])
        Y[:, 1] = 2*c[:, 0]*np.sqrt(X[:, m-1]*(1-X[:, m-1]))*np.cos(X[:, m] * 2 * np.pi)
        Y[:, 2] = 2*c[:, 0]*np.sqrt(X[:, m-1]*(1-X[:, m-1]))*np.sin(X[:, m] * 2 * np.pi)
        for l in range(m-1):
            Y[:, 2*l+3] = c[:, l+1]*np.cos(X[:, m+l+1] * 2 * np.pi)
            Y[:, 2*l+4] = c[:, l+1]*np.sin(X[:, m+l+1] * 2 * np.pi)
    else:
        m = dim//2
        g = np.zeros((n, m+1))
        c = np.zeros((n, m))
        g[:, 0] = 0
        g[:, m] = 1
        for j in range(m-1):
            g[:, m-j-1] = g[:, m-j]*np.power(X[:, m-j-2],1/(m-j-1))
            c[:, m-j-1] = np.sqrt(g[:, m-j]-g[:, m-j-1])
        c[:, 0] = np.sqrt(g[:, 1] - g[:, 0])
        Y = np.zeros((n, d))
        for l in range(m):
            Y[:, 2*l] = c[:, l]*np.cos(X[:, m+l-1] * 2 * np.pi)
            Y[:, 2*l+1] = c[:, l]*np.sin(X[:, m+l-1] * 2 * np.pi)
    return Y

def sct_sphere_points(n, d):
    import qmcpy
    X = qmcpy.Sobol(d - 1, graycode=True).gen_samples(n)
    n = X.shape[0]
    Y = np.ones((n, d))
    for i in range(n):
        Y[i][0] *= np.sin(X[i, 0] * 2 * np.pi)
        Y[i][1] *= np.cos(X[i, 0] * 2 * np.pi)

    for j in range(2, d):
        inv_beta = 1 / beta(j / 2, 1 / 2)
        for i in range(n):
            root_function = lambda varphi: inv_beta * int_sin_m(varphi, j - 1) - X[i, j - 1]
            deg = root_scalar(root_function, bracket=[0, np.pi], xtol=1e-15).root
            for k in range(j):
                Y[i][k] *= np.sin(deg)
            Y[i][j] *= np.cos(deg)
    return Y

def gen_sobol_permutations(n, d, method):
    if method == "bm":
        sphere_points = bm_sphere_points(n, d - 1)
    elif method == "tfww":
        sphere_points = tfww_sphere_points(n, d - 1)
    else:
        sphere_points = sct_sphere_points(n, d - 1)
    basis = zero_sum_projection(d)
    projected_sphere_points = sphere_points.dot(basis)
    p = np.zeros((n, d), dtype=np.int64)
    for i in range(n):
        p[i] = np.argsort(projected_sphere_points[i])
    return p

In [6]:
## set the parameters
beta_gibbs = pd.read_csv('data/beta_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
v2_gibbs = pd.read_csv('data/v2_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None,
                        dtype=np.float64)
the_R = pd.read_csv('data/the_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
tau2_R = pd.read_csv('data/tau2_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
kap_R = pd.read_csv('data/kap_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
lam_R = pd.read_csv('data/lam_R_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
mu = pd.read_csv('data/mu_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)
sd = pd.read_csv('data/sd_s5a1-R15-explore0.3-v1-modelrisk--ntime36-sigma10.txt', header=None, dtype=np.float64)

bn_post = bayesian_network_posterior(the_R.to_numpy(), tau2_R.to_numpy(), kap_R.to_numpy(), lam_R.to_numpy(),
                                         mu.to_numpy().flatten(), 5, 1, 36, beta_gibbs, v2_gibbs)
# p_v2 = bn_post.posterior_sample(1, useFixed=False)[1]
mua = np.array(mu.to_numpy().reshape(36, 6)[:, 0]).reshape(1, 36)
mus = np.array(mu.to_numpy().reshape(36, 6)[:, 1:]).reshape(5, 36)
# v2_s0 = np.array(p_v2.reshape(36, 6)[:, 1:]).reshape(5, 36)

theta0 = np.load('theta.npy')
theta = np.zeros((5, 13))
theta_index = [0, 1, 3, 6, 7, 9, 12, 14, 18, 20, 24, 26, 30]
for i in range(13):
    t = theta_index[i]
    theta[:, i] = theta0[:, t]
s1 = np.array([0.05, 0, 0, 30, 5, 0.7])


In [7]:
def QMC_theta(s1, theta, mua, mus, mu, sd, params, v2_s0, process_model, P, in_size = 10):
    # calculate approximate shapley value using QMC
    s_index = [0, 1, 3, 6, 7, 9, 12, 14, 18, 20, 24, 26, 30, 35]
    [process_model.m_s, process_model.r_l, process_model.beta_LC_max, process_model.mu_max] = params
    v2_s = np.zeros((5, 14))
    for i in range(14):
        t = s_index[i]
        v2_s[:, i] = v2_s0[:, t]
    m = np.shape(P)[0]
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1] + 1
    num_th = nums * (H - 1)
    mean = np.zeros((nums, H))
    delta_s_mean = np.zeros((m, num_th, nums, H))
    delta_s_var = np.zeros((m, num_th, nums, H))
    delta_r_mean = np.zeros((m, num_th))
    delta_r_var = np.zeros((m, num_th))
    theta_flat = theta.flatten()
    for i in range(m):
        permute = P[i, :]
        temp = np.zeros(num_th)
        [r_prev, s_prev] = simulate(s1, temp.reshape(nums, H - 1), mua, mus, mu, sd, mean, v2_s, process_model, in_size = 10)
        [mean_r_prev, var_r_prev] = [np.mean(r_prev), np.var(r_prev)]
        [mean_s_prev, var_s_prev] = [np.mean(s_prev, axis = 0), np.var(s_prev, axis = 0)]
        for j in range(num_th):
            index = permute[j]
            temp[index] = theta_flat[index].copy()
            [r, s] = simulate(s1, temp.reshape(nums, H - 1), mua, mus, mu, sd, mean, v2_s, process_model, in_size = 10)
            [mean_r, var_r] = [np.mean(r), np.var(r)]
            [mean_s, var_s] = [np.mean(s, axis = 0), np.var(s, axis = 0)]         
            delta_s_mean[i, index, :, :] = mean_s - mean_s_prev
            delta_s_var[i, index, :, :] = var_s - var_s_prev
            delta_r_mean[i, index] = mean_r - mean_r_prev
            delta_r_var[i, index] = var_r - var_r_prev
            [mean_r_prev, var_r_prev] = [mean_r.copy(), var_r.copy()]
            [mean_s_prev, var_s_prev] = [mean_s.copy(), var_s.copy()]
    shap_s_mean = np.mean(delta_s_mean, axis = 0)
    shap_s_var = np.mean(delta_s_var, axis = 0)
    shap_r_mean = np.mean(delta_r_mean, axis = 0)
    shap_r_var = np.mean(delta_r_var, axis = 0)
    return [shap_s_mean, shap_s_var, shap_r_mean, shap_r_var]


def QMC_random_mean(s1, theta, mua, mus, mu, sd, params, v2_s0, process_model, P, in_size = 10):
    # calculate approximate shapley value using QMC
    s_index = [0, 1, 3, 6, 7, 9, 12, 14, 18, 20, 24, 26, 30, 35]
    [process_model.m_s, process_model.r_l, process_model.beta_LC_max, process_model.mu_max] = params
    v2_s = np.zeros((5, 14))
    for i in range(14):
        t = s_index[i]
        v2_s[:, i] = v2_s0[:, t]
    m = np.shape(P)[0]
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1] + 1
    mean = v2_s/100
    num_factor = nums * H 
    delta_s = np.zeros((m, nums * H, nums, H))
    delta_r = np.zeros((m, nums * H))
    k = 0
    mean_flat = mean.flatten()
    v2_s_flat = v2_s.flatten()
    for i in range(m):
        permute = P[i, :]
        temp_mean = np.zeros(nums * H)
        temp_v2_s = v2_s_flat.copy()
        [r_prev, s_prev] = simulate(s1, theta, mua, mus, mu, sd, temp_mean.reshape(nums, H), temp_v2_s.reshape(nums, H), process_model, in_size = 10)
        [mean_r_prev, mean_s_prev] = [np.mean(r_prev), np.mean(s_prev, axis = 0)]
        for j in range(nums * H):
            index = permute[j]
            temp_mean[index] = mean_flat[index].copy()
            temp_v2_s[index] = 0
            [r, s] = simulate(s1, theta, mua, mus, mu, sd, temp_mean.reshape(nums, H), temp_v2_s.reshape(nums, H), process_model, in_size = 10)
            [mean_r, mean_s] = [np.mean(r), np.mean(s, axis = 0)]
            delta_s[i, index, :, :] = mean_s - mean_s_prev
            delta_r[i, index] = mean_r - mean_r_prev
            [mean_r_prev, mean_s_prev] = [mean_r.copy(), mean_s.copy()]
    shap_s = np.mean(delta_s, axis = 0)
    shap_r = np.mean(delta_r, axis = 0)
    return [shap_s, shap_r]

def QMC_random_var(s1, theta, mua, mus, mu, sd, params, v2_s0, process_model, P, mid_size = 10, in_size = 10):
    # calculate approximate shapley value using QMC
    s_index = [0, 1, 3, 6, 7, 9, 12, 14, 18, 20, 24, 26, 30, 35]
    [process_model.m_s, process_model.r_l, process_model.beta_LC_max, process_model.mu_max] = params
    v2_s = np.zeros((5, 14))
    for i in range(14):
        t = s_index[i]
        v2_s[:, i] = v2_s0[:, t]
    m = np.shape(P)[0]
    nums = np.shape(theta)[0]
    H = np.shape(theta)[1] + 1
    mean = np.zeros((nums, H))
    delta_s = np.zeros((m, nums * H, nums, H))
    delta_r = np.zeros((m, nums * H))
    mean_flat = mean.flatten()
    v2_s_flat = v2_s.flatten()
    for i in range(m):
        permute = P[i, :]
        temp_mean = mean_flat.copy()
        temp_v2_s = np.zeros(nums * H)
        temp_v2_s_fix = v2_s_flat.copy()
        [r_prev, s_prev] = simulate(s1, theta, mua, mus, mu, sd, temp_mean.reshape(nums, H), temp_v2_s.reshape(nums, H), process_model, in_size = 10)
        [var_r_prev, var_s_prev] = [np.var(r_prev), np.var(s_prev, axis = 0)]
        for j in range(nums * H):
            index = permute[j]
            temp_v2_s[index] = v2_s_flat[index].copy()
            temp_v2_s_fix[index] = 0
            var_r = 0
            var_s = np.zeros((nums, H))
            for k in range(mid_size):
                temp_mean = np.random.normal(loc = mean_flat.copy(), scale = temp_v2_s_fix / 100)
                [r, s] = simulate(s1, theta, mua, mus, mu, sd, temp_mean.reshape(nums, H), temp_v2_s.reshape(nums, H), process_model, in_size = 10)
                [var_r, var_s] = [var_r + np.var(r), var_s + np.var(s, axis = 0)]
            [var_r, var_s] = [var_r / mid_size, var_s / mid_size]
            delta_s[i, index, :, :] = var_s - var_s_prev
            delta_r[i, index] = var_r - var_r_prev
            [var_r_prev, var_s_prev] = [var_r.copy(), var_s.copy()]
    shap_s = np.mean(delta_s, axis = 0)
    shap_r = np.mean(delta_r, axis = 0)
    return [shap_s, shap_r]

In [9]:
# shapley value of theta
np.random.seed(1234)
ssize1 = 200
m1 = 500
p_beta, p_v2, mu0 = bn_post.posterior_sample(10000, useFixed=False)
v2_s = np.array(np.mean(p_v2, 1).reshape(36, 6)[:, 1:]).reshape(5, 36)/5
start = time.time()
nums = 5
H = 14


# sample the model parameters
param_mean = np.tile(np.array([0.0225, 0.4792, 0.1426, 0.3845]), (ssize1, 1))
param_std = 0.1*param_mean
params = np.random.normal(loc = param_mean, scale = param_std)

# sample the permutations
P = np.zeros((ssize1, m1, nums * (H - 1)), dtype=np.int64)
for i in range(ssize1):
    P_i = gen_sobol_permutations(m1, nums * (H - 1), "tfww")
    P[i, :, :] = P_i

# calculation
def cal_theta(s1, theta, mua, mus, mu, sd, params, v2_s, P, in_size = 10):
    process_model = mechanism()
    [a, b, c, d] = QMC_theta(s1, theta, mua, mus, mu, sd, params, v2_s, process_model, P)
    return [a, b, c, d]

delta_list1 = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize1):
    delta_list1.append(pool.apply_async(cal_theta, args = (s1, theta, mua, mus, mu, sd, params[i, :], v2_s, P[i, :, :])))
pool.close()
pool.join()   
end = time.time()
print(end - start)

shap_s_mean_theta_samples = np.zeros((ssize1, nums * (H - 1), nums, H))
shap_s_var_theta_samples = np.zeros((ssize1, nums * (H - 1), nums, H))
shap_r_mean_theta_samples = np.zeros((ssize1, nums * (H - 1)))
shap_r_var_theta_samples = np.zeros((ssize1, nums * (H - 1)))
for i in range(ssize1):
    shap_s_mean_theta_samples[i, :, :, :] = delta_list1[i].get()[0]
    shap_s_var_theta_samples[i, :, :, :] = delta_list1[i].get()[1]
    shap_r_mean_theta_samples[i, :] = delta_list1[i].get()[2]
    shap_r_var_theta_samples[i, :] = delta_list1[i].get()[3]

np.save('nonlinear_shap_s_mean_theta_samples.npy', shap_s_mean_theta_samples)
np.save('nonlinear_shap_s_var_theta_samples.npy', shap_s_var_theta_samples)
np.save('nonlinear_shap_r_mean_theta_samples.npy', shap_r_mean_theta_samples)
np.save('nonlinear_shap_r_var_theta_samples.npy', shap_r_var_theta_samples)    


2824.157334089279


In [10]:
# predictive shapley value of e
np.random.seed(1234)
ssize2 = 200
m2 = 500
p_beta, p_v2, mu0 = bn_post.posterior_sample(10000, useFixed=False)
v2_s = np.array(np.mean(p_v2, 1).reshape(36, 6)[:, 1:]).reshape(5, 36)/5
start = time.time()
nums = 5
H = 14

# sample the model parameters
param_mean = np.tile(np.array([0.0225, 0.4792, 0.1426, 0.3845]), (ssize2, 1))
param_std = 0.1*param_mean
params = np.random.normal(loc = param_mean, scale = param_std)

# sample the permutations
P = np.zeros((ssize2, m2, nums * H), dtype=np.int64)
for i in range(ssize2):
    P_i = gen_sobol_permutations(m2, nums * H, "tfww")
    P[i, :, :] = P_i

# calculation
def cal_mean_e(s1, theta, mua, mus, mu, sd, params, v2_s, P, in_size = 10):
    process_model = mechanism()
    [a, b] = QMC_random_mean(s1, theta, mua, mus, mu, sd, params, v2_s, process_model, P)
    return [a, b]

delta_list2 = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize2):
    delta_list2.append(pool.apply_async(cal_mean_e, args = (s1, theta, mua, mus, mu, sd, params[i, :], v2_s, P[i, :, :])))
pool.close()
pool.join()   
end = time.time()
print(end - start)

shap_s_mean_e_samples = np.zeros((ssize2, nums * H, nums, H))
shap_r_mean_e_samples = np.zeros((ssize2, nums * H))
for i in range(ssize2):
    shap_s_mean_e_samples[i, :, :, :] = delta_list2[i].get()[0]
    shap_r_mean_e_samples[i, :] = delta_list2[i].get()[1]
    
np.save('nonlinear_shap_s_mean_e_samples.npy', shap_s_mean_e_samples)
np.save('nonlinear_shap_r_mean_e_samples.npy', shap_r_mean_e_samples)

2989.2124297618866


In [11]:
# variance based shapley value of e
np.random.seed(1234)
ssize3 = 200
m3 = 500
p_beta, p_v2, mu0 = bn_post.posterior_sample(10000, useFixed=False)
v2_s = np.array(np.mean(p_v2, 1).reshape(36, 6)[:, 1:]).reshape(5, 36)/5
start = time.time()
nums = 5
H = 14

# sample the model parameters
param_mean = np.tile(np.array([0.0225, 0.4792, 0.1426, 0.3845]), (ssize3, 1))
param_std = 0.1*param_mean
params = np.random.normal(loc = param_mean, scale = param_std)

# sample the permutations
P = np.zeros((ssize3, m3, nums * H), dtype=np.int64)
for i in range(ssize3):
    P_i = gen_sobol_permutations(m3, nums * H, "tfww")
    P[i, :, :] = P_i

# calculation
def cal_var_e(s1, theta, mua, mus, mu, sd, params, v2_s, P):
    process_model = mechanism()
    [a, b] = QMC_random_var(s1, theta, mua, mus, mu, sd, params, v2_s, process_model, P)
    return [a, b]

delta_list3 = []
pool = mp.Pool(mp.cpu_count())
for i in range(ssize3):
    delta_list3.append(pool.apply_async(cal_var_e, args = (s1, theta, mua, mus, mu, sd, params[i, :], v2_s, P[i, :, :])))
pool.close()
pool.join()   
end = time.time()
print(end - start)

shap_s_var_e_samples = np.zeros((ssize3, nums * H, nums, H))
shap_r_var_e_samples = np.zeros((ssize3, nums * H))
for i in range(ssize3):
    shap_s_var_e_samples[i, :, :, :] = delta_list3[i].get()[0]
    shap_r_var_e_samples[i, :] = delta_list3[i].get()[1]

np.save('nonlinear_shap_s_var_e_samples.npy', shap_s_var_e_samples)
np.save('nonlinear_shap_r_var_e_samples.npy', shap_r_var_e_samples)

26756.274783611298
